<a href="https://colab.research.google.com/github/jamelof23/ASL2/blob/main/asl100_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install compatible versions of dependent libraries

In [ ]:
# Uninstall existing packages that may cause conflicts
!pip uninstall -y tensorflow tensorflow-metadata

# Install specific compatible versions
!pip install tensorflow==2.11.0
!pip install mediapipe==0.10.15
!pip install protobuf==3.20.3

import tensorflow as tf
import mediapipe as mp

print("TensorFlow version:", tf.__version__)
print("MediaPipe version:", mp.__version__)


# Upload dataset

Upload archive.zip to /content/sample_data/

# Unzip the Dataset

In [ ]:
!unzip /content/sample_data/archive.zip -d /content/asl_alphabet

# Detect hand landmarks in training images using mediapipe

In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np
from google.colab.patches import cv2_imshow

# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.2)

# Define directories
train_dir = '/content/asl_alphabet/asl_alphabet_train/asl_alphabet_train/'
output_dir = '/content/asl_alphabet/hand_landmarks/'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# To track the first image processed for each class
saved_classes = set()
output_summary = []

# Process images in the training directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)

    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)

        # Read the image
        image = cv2.imread(image_path)

        # Check if the image is read correctly
        if image is None:
            print(f"Error reading image: {image_path}")
            continue

        # Convert image to RGB for MediaPipe processing
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process the image to find hands
        results = hands.process(image_rgb)

        # Check if hands are detected
        if results.multi_hand_landmarks:
            # Store the landmarks
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the image for visualization
                mp.solutions.drawing_utils.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Create a list of landmarks
                landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
                landmarks_array = np.array(landmarks).flatten()  # Flatten the landmarks

                # Save landmarks as a numpy array
                output_file = os.path.join(output_dir, f"{class_name}_{image_name}.npy")
                np.save(output_file, landmarks_array)

            # Save the first image of each class with detected hands
            if class_name not in saved_classes:
                print(class_name)  # Add this line to print the class name
                cv2_imshow(image)  # Display the image
                cv2.waitKey(1)  # Show the image briefly without waiting for key press
                saved_classes.add(class_name)  # Mark this class as processed

            output_summary.append(f"{image_name}: Hands detected")

        else:
            output_summary.append(f"{image_name}: No hands detected")

# Write summary to a file
with open('output_summary.txt', 'w') as f:
    for line in output_summary:
        f.write(line + "\n")

# Release the MediaPipe hands object
hands.close()


# Prepare Dataset

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Define directories
landmark_dir = '/content/asl_alphabet/hand_landmarks/'

# Define class labels
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'nothing', 'space', 'delete']

# Prepare data and labels
X = []
y = []

for label in labels:
    class_files = [f for f in os.listdir(landmark_dir) if f.startswith(label)]

    for file_name in class_files:
        file_path = os.path.join(landmark_dir, file_name)
        landmarks = np.load(file_path)
        X.append(landmarks)
        y.append(label)

X = np.array(X)
y = np.array(y)

# Print shape of data
print(f"Shape of X (landmarks): {X.shape}")
print(f"Shape of y (labels): {y.shape}")

# Print first few samples to validate
print("\nSample landmarks from X:")
print(X[:3])  # Print the first 3 samples of X

print("\nSample labels from y:")
print(y[:3])  # Print the first 3 labels from y

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Print shapes after splitting
print(f"\nShape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_val: {X_val.shape}")
print(f"Shape of y_val: {y_val.shape}")

# Print number of samples per class in the training set
from collections import Counter
print("\nNumber of samples per class in training set:")
print(Counter(y_train))


# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
label_encoder = LabelEncoder()

# Fit and transform the training and validation labels
y_train_enc = label_encoder.fit_transform(y_train)
y_val_enc = label_encoder.transform(y_val)

# Encode the labels
y_train_onehot = to_categorical(y_train_enc, num_classes=len(labels))
y_val_onehot = to_categorical(y_val_enc, num_classes=len(labels))

# Define the Neural Network Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.layers import BatchNormalization
#Define the model
model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],), activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(len(labels), activation='softmax')
])

# Compile the Model

In [ ]:
#Set up early stopping
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Training the Model

In [ ]:
# Train the model
history = model.fit(X_train, y_train_onehot,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val_onehot),
                    callbacks=[early_stopping])

# Print summary of the model training
print("Model training complete.")



# Saving the Model

In [ ]:
# Save the trained model
model_save_path = '/content/asl_hand_landmarks_model.h5'
model.save(model_save_path)

print(f"Model saved to {model_save_path}")
